In [1]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

from tensorflow.examples.tutorials.mnist import input_data

c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = n

In [2]:
def Shortcut(input_data, fx, data_format = "channels_first"):

  # チャンネル数の取得
  if data_format == "channels_first":
    channel_num = input_data.shape[1]
  else:
    channel_num = input_data.shape[3]

  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  buf = Convolution2D(channel_num, (1,1), strides=(1,1), padding='valid', data_format=data_format)(input_data)

  # 2つを足す
  return add([buf, fx])

In [3]:
def Resblock(n_filters, strides=(1,1), data_format = "channels_first"):
  def f(input_data):    
    fx = Convolution2D(n_filters, (3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
    fx = BatchNormalization()(fx)
    fx = Activation('relu')(fx)
    fx = Convolution2D(n_filters, (3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(fx)
    fx = BatchNormalization()(fx)

    return Shortcut(input_data, fx, data_format)

  return f

In [12]:
def Resnet():
  channelss_first = True
  
  input_data = Input(shape=(1, 28, 28))
  x = Convolution2D(32, (7,7), strides=(1,1), kernel_initializer='he_normal', padding='same')(input_data)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3, 3), strides=(2,2), padding='same')(x)


  x = Resblock(n_filters=64)(x)
#  x = Resblock(n_filters=64)(x)
#  x = Resblock(n_filters=64)(x)
#  x = MaxPooling2D(strides=(2,2))(x)  
#  x = Resblock(n_filters=128)(x)
#  x = Resblock(n_filters=128)(x)
#  x = Resblock(n_filters=128)(x)


  x =  GlobalAveragePooling2D()(x)
  x = Dense(10, kernel_initializer='he_normal', activation='softmax')(x)

  model = Model(inputs=input_data, outputs=x)
  return model

In [13]:
model = Resnet()
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

TypeError: Failed to convert object of type <class 'tuple'> to Tensor. Contents: (1, 1, 1, Dimension(1)). Consider casting elements to a supported type.